In [1]:
import glob
import json, os
import cv2 as cv
import numpy as np
import pandas as pd

c:\Users\Legion\.conda\envs\tf210\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Legion\.conda\envs\tf210\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Legion\.conda\envs\tf210\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
raw_data_dir = 'data/raw/'
processed_data_dir = 'data/processed/'

In [3]:
def create_masks():
    all_images = glob.glob(raw_data_dir + '*/*.jpg')
    for image_path in all_images:
        image_path = image_path.replace('\\', '/')
        mask_json_path = image_path.replace('.jpg', '.json')
        if os.path.exists(image_path) and os.path.exists(mask_json_path):
            # print("Processing image: ", image_path)
            mask_json = json.load(open(mask_json_path))
            image_height = mask_json['imageHeight']
            image_width = mask_json['imageWidth']

            mask = np.zeros((image_height, image_width), dtype=np.float32)
            for shape in mask_json['shapes']:
                points = np.array(shape['points'], dtype=np.int32)
                label = shape['label']  
                if label == 'leaf':
                    cv.fillPoly(mask, [points], 255)
                elif label == 'spot':
                    cv.fillPoly(mask, [points], 128)
                else:
                    cv.fillPoly(mask, [points], 0)

            image_path_new = image_path.replace(raw_data_dir, processed_data_dir + 'images/')
            mask_path_new = image_path.replace(raw_data_dir, processed_data_dir + 'masks/').replace('.jpg', '.png')

            os.makedirs(os.path.dirname(image_path_new), exist_ok=True)
            os.makedirs(os.path.dirname(mask_path_new), exist_ok=True)

            # print("Saving image: ", image_path_new)
            # print("Saving mask: ", mask_path_new)
            # print('\n')
            cv.imwrite(image_path_new, cv.imread(image_path))
            cv.imwrite(mask_path_new, mask)

In [4]:
create_masks()